In [ ]:
# This jnb script is used to convert V0 B0 B1 defined (directly download from Cottenier's webpage) and put as string as module `calculations/wien2k_ref.py`
# to the json format from ACWF and used in workflow.
# The unit of B0 is differnt where `wien2k_ref.py` is GPa while ACWF use eV/ang^3.
# Therefore for B0 value fro wien2k_ref / 160.21766208 is value in ACWF
# Moreover, the V0 in wien2k_ref is value per atom while not per atom in ACWF. Here I will recover to not per atom.

In [1]:
import json

from aiida_sssp_workflow.calculations.wien2k_ref import WIEN2K_REF, WIEN2K_REN_REF
from aiida_sssp_workflow.utils import RARE_EARTH_ELEMENTS

In [2]:
GPA2EVANG = 160.21766208

In [3]:
def helper_get_v0_b0_b1(element: str, num_atom: int):
    """get eos reference of element"""
    import re

    # import ipdb; ipdb.set_trace()

    if element in RARE_EARTH_ELEMENTS:
        element_str = f"{element}N"
    else:
        element_str = element

    regex = re.compile(
        rf"""{element_str}\s+
                        (?P<V0>\d*.\d*)\s*
                        (?P<B0>\d*.\d*)\s*
                        (?P<B1>\d*.\d*)""",
        re.VERBOSE,
    )
    if element not in RARE_EARTH_ELEMENTS:
        match = regex.search(WIEN2K_REF)
        V0 = match.group("V0")
        B0 = match.group("B0")
        B1 = match.group("B1")
    else:
        match = regex.search(WIEN2K_REN_REF)
        V0 = match.group("V0")
        B0 = match.group("B0")
        B1 = match.group("B1")

    return float(V0) * num_atom, float(B0) / GPA2EVANG, float(B1)

In [4]:
def cifurl2numatom(url) -> int:
    import requests
    import tempfile
    from pymatgen.io.cif import CifParser

    with tempfile.NamedTemporaryFile(suffix=".cif") as tmpf:
        response = requests.get(url)
        data = response.text

        tmpf.write(str.encode(data))
        tmpf.flush()
        # print(data)
        # print(tmpf.name)

        # import ipdb; ipdb.set_trace()
        cifparser = CifParser(filename=tmpf.name)
        structure = cifparser.get_structures(primitive=False)[0]
        numatom = structure.num_sites

    return numatom

In [7]:
elements = [
    "Ag",
    "Al",
    "Ar",
    "As",
    "At",
    "Au",
    "Ba",
    "Be",
    "B",
    "Bi",
    "Br",
    "Ca",
    "Cd",
    "Ce",
    "C",
    "Cl",
    "Co",
    "Cr",
    "Cs",
    "Cu",
    "Dy",
    "Er",
    "Eu",
    "Fe",
    "F",
    "Ga",
    "Gd",
    "Ge",
    "He",
    "Hf",
    "H",
    "Hg",
    "Ho",
    "I",
    "In",
    "Ir",
    "K",
    "Kr",
    "La",
    "Li",
    "Lu",
    "Mg",
    "Mn",
    "Mo",
    "Na",
    "Nb",
    "Nd",
    "Ne",
    "N",
    "Ni",
    "O",
    "Os",
    "Pb",
    "Pd",
    "P",
    "Pm",
    "Po",
    "Pr",
    "Pt",
    "Rb",
    "Re",
    "Rh",
    "Rn",
    "Ru",
    "Sb",
    "Sc",
    "Se",
    "S",
    "Si",
    "Sm",
    "Sn",
    "Sr",
    "Ta",
    "Tb",
    "Tc",
    "Te",
    "Ti",
    "Tl",
    "Tm",
    "V",
    "W",
    "Xe",
    "Yb",
    "Y",
    "Zn",
    "Zr",
]

res_d_typical = {
    "_comment": "WIEN2k version 13.1 calcDelta package version 3.1 -- calculations by Stefaan Cottenier",
    "BM_fit_data": {},
}
res_d_lann = {
    "_comment": "Rare-Earth Nitrids(REN) WIEN2K eos data (courtesy of M. Topsakal)",
    "BM_fit_data": {},
}
for element in elements:
    # urltociffile

    if element in RARE_EARTH_ELEMENTS:
        V0, B0, B1 = helper_get_v0_b0_b1(element=element, num_atom=1)
        res_d_lann["BM_fit_data"][f"{element}N"] = {
            "bulk_deriv": B1,
            "bulk_modulus_ev_ang3": B0,
            "min_volume": V0 * 2,  # two atom
        }
    else:
        try:
            url = f"https://raw.githubusercontent.com/aiidateam/aiida-sssp-workflow/v22.04.0a0/aiida_sssp_workflow/statics/cif/typical/{element}.cif"
            numatom = cifurl2numatom(url)
        except:
            continue

        V0, B0, B1 = helper_get_v0_b0_b1(element=element, num_atom=numatom)
        res_d_typical["BM_fit_data"][element] = {
            "bulk_deriv": B1,
            "bulk_modulus_ev_ang3": B0,
            "min_volume": V0,
        }

In [8]:
with open("./AE_EOS/WIEN2K_LANN.json", "w") as f:
    json.dump(res_d_lann, f, indent=4)

with open("./AE_EOS/WIEN2K_TYPICAL.json", "w") as f:
    json.dump(res_d_typical, f, indent=4)